# 目的
毎日のOHLCVをダウンロードする。   
CryptoCompare(https://min-api.cryptocompare.com/) では七日間だけminute OHLCV が残されているらしい。   
デイリーの分も残すが、minute がデイリーに及ぼす影響も見ておきたい。


In [ ]:
import urllib.request
import urllib.parse
import json

def getRateViaCrypto(histoticks, params):
    """getRateViaCrypto(histoticks, params)
    get rate via CryptoCompare (https://min-api.cryptocompare.com/).
    params should be a dict object with items of str.
    <params>
    histoticks: day, hour, minute or some tick (see the site)
    params: parameters to send the site 
        fsym : currency symbol of interest (required)
        tsym : currency symbol to convert into (required)
        limit: limit of retrieved data (max: 2000)
        e    : the exchange to obtain data from (CCCAGG - by default)
        toTs : last unix timestamp to return data for
    </params>
    
    <return>
    retrieved data (json object)
    </return>
    """
    
    url = "https://min-api.cryptocompare.com/data/"
    url += "histo{}".format(histoticks) + "?" + urllib.parse.urlencode(params) 
    res = urllib.request.urlopen(url)
    result = json.loads(res.read().decode('utf-8'))
    return result

In [ ]:
import numpy as np
import pandas as pd
import copy
import datetime

datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

def toDataFrame(data, is_datetime=False):
    """toDataFrame(data)
    convert data to a DataFrame object.
    'data' is obtained from CryptoCompare.
    The values in the 'time' are converted to datetime strings 
    if 'is_datetime' == True.
    
    <params>
    data       : data obtained from CryptoCompare (dict object)
    is_datetime: boolean
    </params>
    
    <return>
    a DataFrame object
    </return>
    """
    
    if not isinstance(data, dict):
        raise TypeError
    keys = data["Data"][0].keys()
    
    output = np.zeros((len(data["Data"]), len(keys)), dtype=object)
    for ii, col in enumerate(data["Data"]):
        if is_datetime:
            buff = copy.deepcopy(col)
            datetime1 = datetime.datetime.fromtimestamp(buff["time"])
            buff["time"] = datetime1.strftime(datetimeFmt)
            output[ii] = np.array([buff[key] for key in keys], dtype=object)
        else:
            output[ii] = np.array([col[key] for key in keys])
        
    return pd.DataFrame(output, columns=list(keys))

## minute OHLCV の取得テスト
`minute`を指定する。   
分足は一度に最大300まで取得できる。

In [ ]:
histoticks = "minute"
params = {
    "fsym": "BTC",
    "tsym": "JPY",
    "limit": "60", # １時間分
    "e": "bitFlyer"
}
result = getRateViaCrypto(histoticks, params)
for key in result.keys():
    if key != "Data":
        print(key, result[key])
    else:
        print(key)
        for data in result[key]:
            print(data)

### ローソク足チャート

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.finance import candlestick_ohlc
%matplotlib inline

df = toDataFrame(result, False)
timestamps = df["time"].as_matrix()


# candle stick
fig, axes = plt.subplots(figsize=(8, 8))
_ = candlestick_ohlc(axes, df[["time", "open", "high", "low", "close"]].values, width=np.diff(timestamps)[0]*0.8)
plt.ylabel("JPY/BTC")
plt.xlabel("time")

In [ ]:
import time

datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

toTs_end = datetime.datetime(2018, 5, 28, 0, 0, 0).timestamp()
toTs_last = datetime.datetime(2018, 6, 2, 0, 0, 0).timestamp()
histoticks = "minute"
hours = 4 # ６回に分けてデータを取得する
limit = int(hours*60 - 1)

for ii in range(3*365*24): # とりあえず３年分に設定
    toTs = toTs_last - ii * hours * 3600
    datetime1 = datetime.datetime.fromtimestamp(toTs)
    print(ii, datetime1.strftime(datetimeFmt))
    if toTs <= toTs_end:
        break
    params = {
        "fsym": "BTC",
        "tsym": "JPY",
        "limit": str(limit),
        "e": "bitFlyer",
        "toTs":str(toTs)
    }
    result = getRateViaCrypto(histoticks, params)
    if ii == 0:
        df = toDataFrame(result, True)
    else:
        df = pd.concat([toDataFrame(result, True), df], ignore_index=True)
    time.sleep(10)
#     if ii == 2:
#         break

In [ ]:
df

In [ ]:
df.to_csv("./test_.csv")

In [ ]:
from google.colab import files

files.download("./test.csv")